In [ ]:
pip install -q yahoo_fin

In [ ]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [ ]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

pd.options.display.max_columns = None

In [ ]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '10b', tickers, False)

In [12]:
addAlloc(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
48,MNST,0.56,0.98,262.42,223.44,51.72,54.66,0.06,0.33,0.78,277.92,171.51,52.80,0.02,0.37,0.72,173.76,199.49,51.51,-0.00,9.466667
31,GTLB,1.82,0.96,433.94,760.90,59.83,67.42,0.13,1.04,0.66,300.62,402.13,76.59,0.28,1.08,0.75,550.21,621.78,53.83,-0.10,9.292308
85,WOLF,2.85,0.93,789.91,1088.30,7.04,14.77,1.10,1.81,0.67,597.81,1054.26,14.74,1.09,1.50,0.71,668.69,802.46,7.98,0.13,9.236364
24,ENPH,5.52,0.94,2240.94,2094.41,71.45,96.74,0.35,3.08,0.76,2432.32,2750.12,93.01,0.30,3.31,0.81,2465.19,2543.19,67.18,-0.06,9.228571
89,ZS,5.16,0.95,2187.01,2225.63,187.38,210.29,0.12,2.92,0.77,1939.50,2254.68,231.00,0.23,2.86,0.78,2675.37,1785.09,189.72,0.01,9.083333
6,BASE,0.65,0.95,218.36,245.40,15.44,17.07,0.11,0.43,0.70,129.96,238.83,16.89,0.09,0.35,0.79,268.73,293.34,14.09,-0.09,9.045455
28,FIVE,3.83,0.94,1204.90,1441.24,106.08,123.00,0.16,2.29,0.73,865.09,1587.01,115.88,0.09,2.27,0.68,1015.38,1341.12,103.76,-0.02,9.025000
44,LYFT,0.88,0.94,177.55,370.87,13.57,15.59,0.15,0.53,0.76,336.12,389.24,16.92,0.25,0.59,0.80,153.21,336.72,12.14,-0.11,9.000000
71,SQ,3.50,0.95,1015.42,1736.54,89.65,97.94,0.09,2.00,0.70,751.29,1290.12,100.27,0.12,2.51,0.79,1128.48,1553.43,81.38,-0.09,8.944444
84,UPWK,0.82,0.94,268.64,284.10,16.44,18.60,0.13,0.59,0.63,144.92,307.48,19.31,0.17,0.50,0.82,184.83,335.30,14.12,-0.14,8.938462


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '11b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
32,VUG,1.90,0.99,1654.78,762.94,404.03,395.44,-0.02,1.46,0.74,1249.86,676.19,414.23,0.03,1.40,0.63,1036.22,430.79,387.95,-0.04,9.400000
23,ONLN,0.67,0.96,182.99,400.53,44.93,48.97,0.09,0.53,0.61,40.05,143.01,47.15,0.05,0.62,0.74,175.88,394.42,43.70,-0.03,9.155556
11,IPO,0.61,0.96,249.98,348.81,46.12,50.39,0.09,0.47,0.67,102.87,108.78,46.53,0.01,0.40,0.74,242.61,408.04,42.92,-0.07,9.155556
45,XLY,1.31,0.95,829.73,596.34,218.10,186.63,-0.14,0.81,0.74,640.50,507.57,215.14,-0.01,0.85,0.71,768.79,786.79,192.34,-0.12,9.142857
43,XLU,0.43,0.97,296.28,231.99,81.45,77.51,-0.05,0.26,0.75,269.96,140.42,80.52,-0.01,0.25,0.68,252.44,129.12,75.53,-0.07,9.100000
9,ICLN,0.23,0.94,84.80,141.16,12.12,14.16,0.17,0.14,0.63,51.46,82.82,14.18,0.17,0.15,0.78,81.96,146.98,11.51,-0.05,9.047059
48,XSW,1.39,0.94,708.10,576.87,194.96,167.81,-0.14,0.75,0.72,535.00,473.47,193.67,-0.01,0.84,0.72,647.14,794.84,165.94,-0.15,8.971429
36,XLC,0.65,0.95,319.51,203.52,96.88,88.96,-0.08,0.38,0.82,293.31,214.83,99.74,0.03,0.36,0.66,214.22,182.18,79.90,-0.18,8.875000
4,DAPP,0.41,0.95,157.20,176.44,18.47,16.97,-0.08,0.26,0.67,99.18,118.68,19.38,0.05,0.21,0.66,82.35,81.45,13.43,-0.27,8.875000
15,IWM,1.45,0.94,721.22,760.05,238.77,216.94,-0.09,0.71,0.76,1005.29,759.06,235.81,-0.01,0.77,0.79,921.38,1100.68,218.26,-0.09,8.733333


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON',
            'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'TELL',
            'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '10b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
44,OMQS,0.20,0.87,51.18,85.75,0.15,0.33,1.17,0.50,0.56,46.45,67.56,0.31,1.06,0.16,0.70,41.02,72.81,0.22,0.42,5.006536
67,STEM,0.66,0.86,131.04,264.47,0.39,1.15,1.93,1.09,0.40,-60.57,112.61,0.87,1.21,0.42,0.78,97.52,185.11,0.61,0.56,4.986285
33,INVZ,0.27,0.89,36.86,87.39,0.76,1.07,0.41,0.49,0.42,-15.86,15.46,0.92,0.22,0.17,0.75,28.69,79.05,0.69,-0.09,4.967001
0,AIRI,0.28,0.89,35.59,85.75,4.65,6.35,0.37,0.63,0.53,13.69,48.41,5.56,0.20,0.20,0.71,35.74,62.50,4.40,-0.05,4.937997
58,RVYL,2.31,0.84,558.31,900.63,1.73,4.68,1.70,3.82,0.41,21.06,498.46,3.66,1.11,1.75,0.58,79.21,599.32,2.75,0.59,4.847059
4,AMSSY,2.47,0.84,275.92,599.07,2.88,5.41,0.88,3.39,0.46,-115.48,203.37,4.45,0.55,1.81,0.76,220.06,672.37,3.46,0.20,4.759358
17,CHGG,1.51,0.83,155.39,470.59,2.11,4.01,0.90,2.17,0.43,-416.28,111.21,3.14,0.49,1.23,0.67,65.44,441.95,2.56,0.22,4.693464
60,SAVE,0.99,0.79,67.10,222.76,0.64,2.11,2.30,1.08,0.52,14.73,85.26,1.11,0.74,0.80,0.74,81.01,237.55,0.78,0.22,4.555754
6,APPS,1.25,0.78,297.29,429.17,1.44,4.10,1.85,2.49,0.46,-106.35,408.22,2.54,0.76,1.03,0.74,193.44,560.12,2.03,0.41,4.469316
59,SABR,0.25,0.87,56.46,85.42,3.91,4.63,0.18,0.43,0.49,-33.17,38.51,4.49,0.15,0.18,0.73,63.64,108.91,3.57,-0.09,4.395425


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '11b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
26,META,6.57,0.96,2908.95,1334.62,574.32,489.70,-0.15,5.64,0.85,2990.33,1822.44,566.25,-0.01,5.79,0.51,1251.88,-569.73,477.47,-0.17,6.590476
0,AAPL,2.38,0.95,1105.64,659.10,237.33,211.70,-0.11,1.25,0.78,1186.61,716.37,231.51,-0.02,1.48,0.65,917.68,446.86,207.12,-0.13,6.331169
29,NKE,1.78,0.93,611.57,819.72,78.77,88.51,0.12,1.12,0.70,426.79,686.09,82.77,0.05,1.17,0.77,566.01,882.91,77.11,-0.02,6.059524
6,BA,3.52,0.94,1156.95,1471.52,155.44,142.86,-0.08,2.06,0.68,868.89,1436.15,155.62,0.00,1.94,0.75,1099.05,1902.53,125.21,-0.19,5.964286
19,FDX,3.67,0.94,1700.08,1069.67,302.67,281.33,-0.07,2.25,0.78,1483.76,1128.60,292.94,-0.03,2.19,0.74,1394.36,892.47,265.20,-0.12,5.857143
34,QCOM,2.30,0.96,1198.84,887.31,158.53,164.54,0.04,1.30,0.74,1162.47,875.98,176.75,0.11,1.54,0.71,919.05,581.40,152.23,-0.04,5.857143
14,DE,5.03,0.89,2141.36,1551.09,465.90,398.97,-0.14,2.62,0.75,2469.23,1889.85,438.88,-0.06,3.45,0.68,2223.95,1436.99,362.80,-0.22,5.571429
22,IBM,1.63,0.96,952.14,423.17,227.41,221.65,-0.03,1.36,0.75,861.70,517.24,223.90,-0.02,1.54,0.63,596.05,197.85,201.39,-0.11,5.523810
25,LLY,8.91,0.89,4583.50,1201.49,795.35,699.81,-0.12,7.61,0.80,4424.53,1344.66,799.30,0.00,9.24,0.55,1012.66,-696.57,612.50,-0.23,5.440476
1,ACN,3.82,0.92,2028.07,1240.53,362.37,335.21,-0.07,2.06,0.79,1696.36,1322.27,364.57,0.01,2.19,0.69,1617.10,1511.78,330.20,-0.09,5.428571


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '11a', tickers5, False)

In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
49,PROSF,1.77,0.88,391.55,760.28,41.10,62.04,0.51,1.65,0.62,92.47,512.55,50.74,0.23,1.13,0.81,392.24,638.45,41.49,0.01,5.815126
53,SFTBY,0.61,0.94,187.99,264.75,30.36,33.87,0.12,0.45,0.64,75.73,146.08,31.84,0.05,0.28,0.77,175.72,239.65,27.26,-0.10,5.714286
3,BAK,0.54,0.90,139.03,152.61,4.82,6.00,0.24,0.42,0.60,108.69,124.61,7.54,0.56,0.34,0.66,68.36,87.80,5.12,0.06,5.595238
1,ADDDF,5.86,0.91,1370.73,1829.06,247.50,291.65,0.18,4.60,0.75,1299.53,1382.80,295.78,0.20,3.61,0.71,1135.19,2060.33,239.72,-0.03,5.500000
23,FANUY,0.32,0.91,71.29,106.67,13.29,14.85,0.12,0.28,0.66,54.22,67.59,15.91,0.20,0.16,0.74,87.76,141.90,12.91,-0.03,5.000000
59,TELNF,0.17,0.89,48.79,71.42,11.44,13.19,0.15,0.12,0.67,35.09,39.44,12.85,0.12,0.10,0.71,31.49,54.02,11.51,0.01,4.890476
18,EQNR,0.65,0.88,165.21,87.18,23.23,25.64,0.10,0.42,0.64,84.34,83.50,27.69,0.19,0.31,0.68,140.85,100.83,23.69,0.02,3.885714
47,PAX,0.17,0.93,71.80,69.23,12.01,12.56,0.05,0.11,0.73,63.21,54.78,13.50,0.12,0.08,0.69,87.67,60.46,12.07,0.01,3.842857
25,FQVLF,0.65,0.91,184.57,261.51,14.37,15.18,0.06,0.43,0.73,212.69,191.67,15.99,0.11,0.36,0.78,209.79,291.95,13.98,-0.03,3.500000
41,NOK,0.08,0.91,25.62,18.52,4.24,4.51,0.06,0.07,0.61,9.77,11.02,4.90,0.16,0.04,0.71,26.94,28.90,3.76,-0.11,3.500000


In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, True)

In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
2,FLN,0.13,0.98,68.79,68.31,16.29,16.87,0.04,0.07,0.74,74.09,73.80,18.09,0.11,0.08,0.69,72.69,50.26,15.17,-0.07,9.300000
14,EWY,0.75,0.94,253.96,329.33,54.96,62.17,0.13,0.39,0.73,252.78,266.14,61.92,0.13,0.52,0.76,223.11,394.60,54.46,-0.01,8.938462
11,EWP,0.23,0.96,120.89,98.90,33.68,31.98,-0.05,0.14,0.75,112.26,64.57,33.26,-0.01,0.19,0.72,83.09,62.26,30.71,-0.09,8.800000
20,KWT,0.17,0.95,116.96,72.74,32.86,35.01,0.07,0.14,0.79,130.14,60.80,32.95,0.00,0.13,0.70,76.27,61.12,32.74,-0.00,8.785714
22,TUR,0.45,0.95,229.45,162.18,37.82,35.33,-0.07,0.35,0.69,180.24,103.35,36.00,-0.05,0.36,0.69,144.35,112.03,31.62,-0.16,8.785714
19,KEMQ,0.36,0.90,72.00,138.90,17.67,24.53,0.39,0.25,0.67,24.00,106.31,20.97,0.19,0.30,0.71,49.74,142.77,17.72,0.00,8.743590
4,EPHE,0.21,0.94,110.88,114.25,26.20,28.19,0.08,0.11,0.72,79.13,84.06,28.29,0.08,0.13,0.73,95.64,119.02,25.74,-0.02,8.650000
0,ARGT,0.69,0.92,431.05,131.40,84.07,73.43,-0.13,0.50,0.76,366.55,131.31,74.90,-0.11,0.59,0.55,182.42,-44.01,67.22,-0.20,8.584615
23,VNM,0.14,0.93,59.00,70.21,11.97,13.04,0.09,0.09,0.70,41.02,42.67,12.76,0.07,0.13,0.60,37.62,58.67,11.93,-0.00,8.522222
7,EWI,0.29,0.93,128.39,112.27,37.76,35.21,-0.07,0.15,0.79,126.22,76.70,36.62,-0.03,0.23,0.62,80.57,56.70,33.75,-0.11,8.300000


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False

tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']
df7 = run_model('val', '11a', tickers7, False)

In [ ]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
22,CI,3.24,0.97,1374.98,1071.50,281.86,304.27,0.08,2.38,0.74,1405.74,889.79,309.73,0.10,2.57,0.74,1636.66,1320.25,275.96,-0.02,9.325000
37,F,0.23,0.93,100.53,117.72,10.39,11.96,0.15,0.17,0.77,108.50,96.78,11.90,0.15,0.12,0.73,77.77,105.18,10.26,-0.01,8.833333
3,AMR,7.62,0.90,2089.24,984.72,228.71,301.46,0.32,4.13,0.87,2786.35,1575.06,246.35,0.08,2.99,0.56,1393.58,685.25,233.22,0.02,8.687500
18,CC,0.59,0.91,252.74,288.41,19.04,22.79,0.20,0.41,0.81,287.59,281.18,23.32,0.23,0.42,0.66,229.87,262.05,19.10,0.00,8.650000
11,BAX,0.84,0.89,212.96,350.40,30.32,42.44,0.40,0.57,0.78,244.58,395.19,40.46,0.33,0.50,0.79,198.66,370.29,31.04,0.02,8.625000
8,ARE,2.00,0.93,670.33,841.20,102.66,112.49,0.10,1.42,0.71,704.08,727.37,118.43,0.15,1.33,0.75,564.37,809.31,100.64,-0.02,8.600000
19,CCI,1.36,0.95,700.52,762.37,97.11,101.58,0.05,1.13,0.73,450.89,719.89,112.95,0.16,0.84,0.65,499.20,378.64,97.22,0.00,8.500000
77,SAFE,1.24,0.92,432.24,686.35,20.43,22.52,0.10,1.01,0.59,132.38,384.37,23.95,0.17,1.00,0.67,280.86,533.24,19.84,-0.03,8.400000
29,COP,1.35,0.95,668.73,556.94,101.02,105.23,0.04,1.22,0.75,638.77,467.43,108.28,0.07,1.00,0.65,634.87,504.15,98.01,-0.03,8.250000
42,FMS,0.39,0.95,149.24,184.32,23.65,24.58,0.04,0.25,0.71,110.93,100.29,25.70,0.09,0.26,0.70,109.19,154.89,23.15,-0.02,8.250000


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
5,GLD,1.68,0.93,642.06,362.03,244.29,224.16,-0.08,1.57,0.74,659.87,229.95,246.26,0.01,1.36,0.68,425.83,231.82,225.11,-0.08,8.425000
8,SLV,0.29,0.95,132.23,107.73,27.75,26.56,-0.04,0.24,0.72,136.39,79.96,29.20,0.05,0.25,0.76,104.09,121.44,26.99,-0.03,8.250000
12,USO,1.26,0.90,432.43,293.03,74.21,68.29,-0.08,0.89,0.72,458.09,424.87,77.31,0.04,1.04,0.79,580.40,395.94,72.13,-0.03,7.750000
4,FXY,0.55,0.90,92.01,206.92,60.05,64.12,0.07,0.45,0.48,-3.23,95.29,63.79,0.06,0.49,0.72,40.69,128.50,60.41,0.01,7.571429
15,WEAT,0.14,0.87,28.40,49.00,4.82,5.21,0.08,0.11,0.50,23.24,35.11,5.56,0.15,0.09,0.76,34.96,38.66,4.99,0.03,7.075000
13,UUP,0.13,0.91,48.33,28.86,30.26,29.08,-0.04,0.10,0.72,49.66,23.11,30.11,-0.01,0.10,0.75,42.37,17.48,28.82,-0.05,6.850000
0,DBC,0.25,0.94,84.85,69.56,22.35,21.91,-0.02,0.17,0.72,92.84,70.64,23.19,0.04,0.20,0.69,93.23,49.20,22.72,0.02,6.400000
11,TLT,1.05,0.90,258.69,398.60,90.15,92.91,0.03,0.82,0.64,177.46,285.62,94.83,0.05,0.82,0.81,229.81,466.83,89.79,-0.00,5.666667
6,ISHG,0.39,0.90,97.21,141.01,70.10,71.60,0.02,0.26,0.64,38.69,62.08,73.17,0.04,0.27,0.80,81.98,128.39,70.07,-0.00,4.000000
9,SOYB,0.28,0.94,90.36,83.15,21.09,21.35,0.01,0.16,0.80,117.62,80.57,21.81,0.03,0.19,0.75,58.01,92.69,20.93,-0.01,3.400000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [14]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
12,NEAR-USD,0.27,0.89,59.25,54.26,5.43,7.20,0.33,0.25,0.59,34.38,45.74,8.32,0.53,0.14,0.71,48.71,28.11,4.69,-0.14,6.023810
11,MATIC-USD,0.05,0.85,9.73,9.24,0.50,1.10,1.18,0.04,0.70,11.55,9.95,0.81,0.61,0.02,0.62,6.58,3.61,0.44,-0.13,5.944310
7,FIL-USD,0.36,0.87,65.21,85.16,5.22,6.94,0.33,0.40,0.74,85.22,87.61,7.39,0.41,0.24,0.65,23.89,62.28,5.20,-0.00,5.820346
14,TON-USD,0.10,0.89,20.52,17.52,1.73,2.07,0.20,0.11,0.76,66.48,68.22,2.19,0.27,0.05,0.73,20.41,17.96,1.80,0.04,5.807143
3,BCH-USD,17.84,0.84,2668.76,2510.84,457.63,747.35,0.63,13.06,0.67,4010.69,2386.50,627.33,0.37,6.77,0.78,2208.45,1687.24,491.28,0.07,5.746032
0,ADA-USD,0.03,0.87,5.41,4.84,0.96,0.74,-0.22,0.04,0.68,6.39,2.40,0.92,-0.04,0.02,0.75,3.91,1.46,0.76,-0.21,5.623377
8,ICP-USD,0.41,0.87,122.29,97.45,10.70,12.32,0.15,0.46,0.63,87.49,98.51,14.37,0.34,0.22,0.65,53.14,23.99,10.54,-0.01,5.347619
5,DOT-USD,0.41,0.82,63.02,63.30,7.41,9.77,0.32,0.34,0.71,72.45,66.65,9.29,0.25,0.18,0.80,60.81,61.85,7.34,-0.01,5.294643
9,LINK-USD,1.18,0.78,149.32,37.02,23.60,18.88,-0.20,1.45,0.53,31.90,-41.26,24.51,0.04,0.47,0.55,35.51,-75.33,18.93,-0.20,4.471429
2,BTC-USD,3426.07,0.78,458154.40,163758.30,97498.33,79945.13,-0.18,2273.20,0.73,469523.28,55708.10,101523.67,0.04,1543.39,0.58,263329.02,-76875.41,79196.87,-0.19,4.349206


In [15]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
ipos-12b-adjclose-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
ipos-12b-high-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
ipos-12b-low-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras

sent 28,829,790 bytes  received 73 bytes  57,659,726.00 bytes/sec
total size is 57,643,394  speedup is 2.00
